## Table of Content:
* [Team](#team)
* [Task](#task)
* [How we approach the challenge](#approach)
* [Our hypotheses/why](#hypotheses)
* [How we solve it](#solve)
* [First approach](#first)
* [Second approach](#second)
* [Pros and cons of our approach and platform](#pros)
* [Acknowledgement](#ackn)
* [Code](#code)
* [Keyword Analysis (Phase 1)](#phase1)
* [Question Search (Phase 2)](#phase2)

<a id="team"></a>
## Team:
#### We are a team of three: one machine learning/NLP scientist, one front-end senior software engineer and one co-op data science intern/engineer. We all work in the same place, Thomson Reuters, and that’s how we know each other.  

<a id="task"></a>
# Task: 
## What do we know about COVID-19 risk factors? 

<a id="approach"></a>
## How we approached the challenge: 
Our platform was built based on the following principles: 
- Empower users (health researchers) to conduct literature survey efficiently. 
- Adaptable to future needs and challenges of health researchers.  
- Modularized to have the capability of being improved and polished in a short time and in parallel.  

<a id="hypotheses"></a>
## Our hypotheses/why? 

AI can benefit researchers by extracting and visualizing information in the most efficient and relevant manner at scale.  In the absence of expert feedback and annotations, we have built/provided a platform that extracts and ranks relevant info with potential for improvement in the future by taking advantage of expert annotation.

<a id="solve"></a>
## How we solve it/why? 

The project was done in two slightly different approaches.  
The first approach focused on developing an end to end pipeline to address the first subtask which was: 
- Data on potential risks factors 
- Smoking, pre-existing pulmonary disease 
- Co-infections (determine whether co-existing respiratory/viral infections make the virus more transmissible or virulent) and other co-    morbidities 
- Neonates and pregnant women 
- Socio-economic and behavioral factors to understand the economic impact of the virus and whether there were differences. 

We have developed a web app to visually accompany the analysis carried out for this task. 
The second approach was focused on question information retrieval but isn’t accompanied by with a web-app yet due to time constraint. 

<a id="first"></a>
## First approach has the following recipe: 
The platform is designed to visualize snippets of relevant topics through the following process: 
- Merge different sources of the data
- Extract all COVID19 related papers 
- Expand and process the list of keywords
- Find excerpts of papers that include the keywords 
- Rank the excerpts
- Create an extractive summary for each paper
- Serve it to the web-app for visualization

<a id="second"></a>
## Second approach has the following recipe: 
The platform is designed to rank snippets of relevant topics through the following process: 

- Merge different sources of the data
- Extract all COVID 19 related paper s
- Convert a question into set of keywords (The questions are extracted from the [medical dictionary](https://docs.google.com/spreadsheets/d/1NoiAFJoydk3zuc-G0qqROarkhaGpfgbQhTVYhbYtLCM/edit#gid=0) shared with Kaggle participants.
- Expand the list of keywords
- Find excerpts of papers that include the keywords 
- Rank the excerpts
- Create an extractive summary for each paper 

<a id="pros"></a>
### Pros and Cons of our approach and platform: 

#### Pros: 

- The code base and the approach is highly modular and simple. It is a well structured baseleine to start.   
- The code is very well documented 
- The visualization resonates with researchers. We interviewed an expert in epidemiology, since that is our target user. He liked all  aspects of it including the extractive summaries of the papers. He also noted that using this platform could reduce the time taken for a literature survey, which normally takes 3 to 4 months, to less than a month.  
- The platform is taking a high recall approach to include everything 
- With a little bit of effort, we can convert this platform to an expert annotation platform where experts can interactively click on irrelevant sentences in the snippets and convert a completely unsupervised approach to a supervised learning task. 

#### Cons: 
- The output wasn't reviewed and validated by a subject matter expert
- Keyword search is not the most efficient search. Word embedding is known to be a better approach for text analysis but there wasn’t enough time to implement and evaluate that. 
- Our ranking follows a simple approach, where an expert should decide on the metric for ranking the importance of a snippet. 
- The second approach is not coming with a visualization. 
- There may be more risk factors than what we currently searched for and an algorithm should extract the unknown risks too. 

<a id="ackn"></a>
### Acknowledgement
We would like to thank Brian Romer (https://www.linkedin.com/in/brianromer/) for his valuable help in designing the visualization as well as Serge Myroshnychenko (https://www.linkedin.com/in/smyroshnychenko/) for his valuable feedback on the interface.

<a id="code"></a>
# Code

## Requirements

In [ ]:
import os
import io
import re
import sys
import glob
import json
import string
import requests

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

stemmer = PorterStemmer()
root = '../input/CORD-19-research-challenge/'
stop_words = list(set(stopwords.words('english')))
stop_words.extend(['within', 'what', 'how', 'eg', 'ie'])

## Data Parsing and Extraction

### `PaperLoader` class will load all papers for the challenge and provide an interface for us to obtain `DataFrames` to work with. The focus will be on:
- Obtaining Paper title, Abstract, Body
    - The text body is filtered to remove sections containing lots of citations and hyperlinks
- Obtaining Authors, Journal of Publication, Publication Date and Publication Date

## Data Parsing and Extraction

### `PaperLoader` class will load all papers for the challenge and provide an interface for us to obtain `DataFrames` to work with. The focus will be on:
- Obtaining Paper title, Abstract, Body
    - The text body is filtered to remove sections containing lots of citations and hyperlinks
- Obtaining Authors, Journal of Publication, Publication Date and Publication Date
- Obtaining journal ratings(H index) to potentially sort paper based on journal quality
    - For the journal ratings, we use a list we obtained from **INSERT LINK HERE**

In [ ]:
class PaperLoader():
    """
    Loads, parses and merges metadata for papers
    """
    
    def __init__(self, root_dir, no_bib=True):
        """
        Initializes PaperLoader class to read all .json files from root_directory
            
            no_bib: if true, clean noisy sections with bibliographies
            root_dir: root directory for papers
        """
        self.ROOT_DIR = root_dir
        self.JSON_FILES = glob.glob(f'{root}/**/*.json', recursive=True)
        self.PAPERS_COLUMN = {
            "doc_id": [None],
            "title": [None],
            "abstract": [None],
            "text_body": [None]
        }
        self.PAPERS_DF = None
        self.NO_BIB = no_bib

    
    def __clean_bib(self, body_text, thres):
        """
        Removes sections with more than 5 URL/DOI/HTTP instances
            
            body_text: array of dictionaries for text_body
            thres: number of hyperlinks tolerated before removal 
        """
        # Sometimes, the text body has duplicate sections consecutively.
        merged_body = []
        for segment in body_text:
            # We will combine these duplicate sections
            if len(merged_body) > 0:
                if merged_body[-1]['section'] == segment['section']:
                    merged_body[-1]['text'] += '\n' + segment['text']
                    continue
            merged_body.append(segment)

        merged_body = [
            segment for segment in merged_body
            if len(re.findall("(http|doi|www)", segment['text'])) <= thres
        ]
        return merged_body


    def create_paper_df(self):
        """
        Creates a Pandas DataFrame from all json files in root_directory
        Each json file represents a paper. 
        Features extracted are: doc_id, title, abstract, text_body
        """
        df_list = []
        
        for i in tqdm(range(len(self.JSON_FILES))):
            file_name = self.JSON_FILES[i]
            
            #Initialize row for returned df. Each row represents a paper
            row = {x: None for x in self.PAPERS_COLUMN}

            with open(file_name) as json_data:
                data = json.load(json_data)

                row['doc_id'] = data['paper_id']
                row['title'] = data['metadata']['title']
                
                # If title is empty, we skip the paper
                if len(row['title']) <= 2:
                    continue

                # If a paper does not have an abstract of a body, we will skip it
                if ('abstract' not in data or 'body_text' not in data):
                    continue
                else:
                    # Now need all of the abstract. Put it all in
                    # a list then use str.join() 
                    abstract_list = [abst['text'] for abst in data['abstract']]
                    abstract = "\n ".join(abstract_list)

                # Skip the paper if abstract is empty
                if len(abstract) <= 2:
                    continue

                row['abstract'] = abstract

                # And lastly the body of the text.
                # These clauses check if the user wants to clean up references
                if self.NO_BIB:
                    body_list = self.__clean_bib(data['body_text'], 4)
                else:
                    body_list = [bt for bt in data['body_text']]

                row['text_body'] = body_list

                df_list.append(row)
        # create final dataframe
        self.PAPERS_DF = pd.DataFrame(df_list)


    def merge_metadata(self, metadata = 'metadata.csv'):
        """
            Joins paper information with information on journal for paper,
            authors, doi and published date  
                metadata: path to csv file containing metadata
        """
        metadata_df = pd.read_csv(self.ROOT_DIR + metadata)
        metadata_df = metadata_df.loc[:, 
                          ['sha', 'publish_time', 'authors', 'journal', 'doi']]
        self.PAPERS_DF = self.PAPERS_DF.merge(metadata_df,
                                              left_on='doc_id',
                                              right_on='sha',
                                              how='inner')


    def get_df(self):
        """
        Returns processed dataframe
        """
        self.PAPERS_DF = self.PAPERS_DF.dropna(
            subset=['abstract', 'text_body'])
        return self.PAPERS_DF

We will now parse the papers from our data(root) directory and store them in `papers_df`.

In [ ]:
paper_loader = PaperLoader(root)
paper_loader.create_paper_df()
paper_loader.merge_metadata()
papers_df = paper_loader.get_df()

In [ ]:
papers_df.head(2)

In [ ]:
papers_df.shape

## Filtering for covid-19 related papers released after 2019
There is a lot of noise in this dataset due to information about other strains of coronavirus so we will select only the papers that are related to Covid-19. 

While the older papers may contain some important insight on the variance among the  different strains of coronavirus, for our purposes, we will only be looking at papers published on 2019 or later because that is when Covid-19 was first discovered in humans.

In [ ]:
# List of keywords for covid-19
cov_list = [
    'novel coronavi',
    'covid',
    'cov_2',
    'cord-19',
    'cord 19',
    '2019-nCoV',
    '2019 ncov',
    '2019 cov',
    'wuhan coronavi',
]

### `RelevantFilter` class will filter the dataframe from `PaperLoader` and filter for covid-19 papers published on 2019 or later. 
We will need to supply a list of covid-related keywords to filter from to the `constructor`


In [ ]:
class RelevantFilter():
    
    def __init__(self, keywords, year='2019'):
        """
        constructor for RelevantFilter
            keywords: keywords to filter for
            year: papers written before this year will be discarded
        """
        self.KEYWORDS = keywords
        self.YEAR = year

    def extract_recent(self, df):
        """
        extracts documents published on or after self.YEAR
        """
        return df[df['publish_time'] >= self.YEAR]

    def filter_papers(self, df):
        """
        Filters for papers whose title have mention of 
        any of the terms in self.KEYWORDS
        """
        pattern = re.compile('(' + "|".join(self.KEYWORDS) + ')',
                                 re.IGNORECASE)
        # We will filter for rows with one or more matches 
        # for title and covid keywords
        df = df[df['title'].apply(lambda x: 
                                  len(pattern.findall(x)) >= 1
                                  if x else False)]
        
        return df

We will filter through `papers_df` to get only covid-19 related papers in `covid_df`

In [ ]:
covid_filter = RelevantFilter(cov_list, '2019')
covid_df = covid_filter.filter_papers(papers_df)
covid_df = covid_filter.extract_recent(covid_df)

In [ ]:
covid_df.shape

In [ ]:
covid_df.head(1)

In [ ]:
list(covid_df.head(2)['title'].values)

<a id="phase1"></a>
## Keyword Analysis (Phase 1)

We will now go through the papers to extract and rank excerpts that contain relevant information about risk factors for covid-19. 
We will do this through an analysis of:

- Risk factors for covid-19
- Study designs
    - We will use this to evaluate the quality of a paper's methodologies for our rankings
- Outcomes
    - We will incentivise excerpts to explicitly mention outcomes that we have found researchers look for(in our interviews)
- Fatality
    - We have determined that information on mortality and fatality would be of high value to researcehrs, and rightly so.

**Note**: The list of keywords were all obtained from a crowdsourced medical dictionary that researchers had assembled. You can find more details [here](https://docs.google.com/spreadsheets/d/1t2e3CHGxHJBiFgHeW0dfwtvCG4x0CDCzcTFX7yz9Z2E/edit#gid=1217643351)

In [380]:
risk_factors = [{
    'name': 'smoking',
    'pattern': '(smoki|smoker|cigar|nicotine|cannabis|marijuana)'
}, {
    'name': 'diabetes',
    'pattern': '(diabet|insulin|blood sugar|blood glucose|ketoacidosis|hyperglycemi)'
}, {
    'name': 'pregnancy',
    'pattern': 'pregnan'
}, {
    'name': 'tuberculosis',
    'pattern': '(tubercul|mtb|\btb[A-Za-z0-9]\b)'
}, {
    'name': 'hypertension',
    'pattern': '(hypertension|blood pressure|hbp)'
}, {
    'name': 'cancer',
    'pattern': 'cancer'
}, {
    'name': 'neonates',
    'pattern': '(baby|neonate|enfant)'
},
    {
    'name': 'liver disease',
    'pattern': 'liver disease'
},{
    'name': 'COPD',
    'pattern': 'COPD'
},{
    'name': 'pulmonary disease',
    'pattern': 'pulm'
},{
    'name': 'race/ethnicity',
    'pattern': 'ethn'
}]

In [381]:
design_list = [
    'mathemat', 'profil', 'cross sectional case control',
    'matched case control', 'contact', 'surviv', 'time to event',
    'time-to-event', 'risk factor analysis', 'logistic regression',
    'cross-sectional case-control', 'matched case-control',
    'observational case series', 'time series analysis', 'survival analysis',
    'investigati', 'model', 'outbreak', 'stochast', 'statist', 'analysi',
    'experiment', 'excret', 'investig',
    'retrospective cohort', 'cross-sectional case-control',
    'cross sectional case control', 'prevalence survey', 'systematic review ',
    'meta-analysis', 'meta analysis', 'matched case-control',
    'matched case control', 'medical record review',
    'observational case series', 'time series analysis',
    'pseudo-randomized controlled', 'pseudo randomized controlled',
    'randomized controlled', 'retrospective analysis', 'retrospective study',
    'retrospective studies'
]

In [382]:
outcome_list = [
    'risk', 'range', 'duration', 'asymptomatic', 'infecti', 'reproducti',
    'route', 'age', 'transmm'
    'stratifi', 'period,', 'health', 'r0', 'shedd', 'viral'
    'period', 'incub', 'generat', 'factor', 'interval,', 'serial'
]

In [383]:
fatality_list = ['icu', 'fatal', 'death', 'die', 'dead', 'dying', 'mortal']

#### The next cell will contain the default coeffecients for the algorithm's prioritization of different features. These coeffecients are a work in progress and we seek to constantly improve them with more expert feedback.

In [402]:
evaluation_weights= {
    'risk': 5,
    'design': 0.5,
    'outcome': 0.5,
    'fatality': 0.5,
    'section': 0.5,
    'inverse_length': 1000
}

### `PaperAnalyzer` class will take in a DataFrame of papers and then analyze each paper. 
The analysis is done with with its `analyze_risks()`, `analyze_designs()` and `analyze_outcomes()` methods that will analyze the risk factors, designs and outcomes respectively for excerpts in the paper. Finally, the `get_df()` method will return a new DataFrame with rankings for relevancy of excerpts. The rankings also factor in the `section` of the paper that the excerpt is from, with sections like **discussion** or **results** that seem to have pertinent, concise information ranked higher. Furthermore, these rankings are also normalized by the lenght of the excerpts

*Note: These rankings for sections were determined through our interviews with epidimiologists.*

#### Helper functions for `PaperAnalyzer` 

In [403]:
def rank_design(design_keyword):
    """
    This helper function ranks study designs. So far we have
    confirmed rankings for only three study designs, but this
    data will be expanded and improved further with time as we 
    speak to more epidimiologists
    """
    design_rankings = {
        'meta': 10,
        'random': 8,
        'pseudo': 6,
    }
    current_ranking = -1
    for key in design_rankings.keys():
        if key in design_keyword.lower():
            current_ranking = min(current_ranking, design_rankings[key])
    
    if current_ranking == -1:
        current_ranking = 4
    return current_ranking    

def flatten(arr):
    """
    Returns a single flat list from a list of lists
    """
    return [item 
            for sublist in arr 
            for item in sublist]

In [404]:
class PaperAnalyzer():    
    """
    Takes in a dataframe of papers and sets it up for analysis
    """
    # Setting up static constants
    DEFAULT_RISKS = risk_factors
    DEFAULT_DESIGNS = design_list
    DEFAULT_OUTCOMES = outcome_list
    DEFAULT_FATAL = fatality_list
    DEFAULT_WEIGHTS = evaluation_weights
    
    def __init__(self, parent_df, weights = None):
        """
        Explodes the passed dataframe on sections for more granular analysis
        Sets up ranks to be updated later by methods. Client can supply their
        own dictionary of weights for different features.
        """
        # Section ratings
        self.section_ratings = {
                        'discus': 10,
                        'concl': 10,
                        'resul': 10,
                        'analy': 9,
                        'impli': 9,
                        'valu': 9,
                        'intro': 6
                        }
        
        parent_df['full_text'] = parent_df['text_body'].apply(lambda x: '\n'.join([sec['text'] for sec in x]))
        self.df = parent_df.explode('text_body')
        # Extracting section headers
        self.df['section'] = self.df['text_body'].apply(lambda x: 
                                                        x['section'] 
                                                        if type(x) == dict 
                                                        else None)
        # Extracting section texts
        self.df['text_body'] = self.df['text_body'].apply(lambda x:
                                                          x['text'] 
                                                          if type(x) == dict 
                                                          else None)
        # Dropping rows where section text is empty
        self.df = self.df[self.df['text_body'].notna()]
        self.df['total_rank'] = 0
        if weights:
            self.weights = weights
        else:
            self.weights = PaperAnalyzer.DEFAULT_WEIGHTS
        # TQDM is used for progress bars
        tqdm.pandas()

    def analyze_risks(self, risk_factors):
        """
        Analyses papers in self.df for risk factors and returns a report df
        with columns has_{risk_factor}?, {risk_factor}_count, 
        {risk_factor}_in_title and updates {total_rank} for each row.
        The match_indices column is produced for ease of visualization
        in the web app.
        """
        if risk_factors == None:
            risk_factors = PaperAnalyzer.DEFAULT_RISKS
        
        if type(risk_factors[0]) == dict:
            patterns = [risk['pattern'] for risk in risk_factors]
        elif type(risk_factors[0]) == str:
            patterns = [risk for risk in risk_factors]
        
        self.df = self.df[self.df['text_body'].apply(lambda x:
                                                    any(re.compile(pattern, re.IGNORECASE).findall(x)
                                                       for pattern in patterns)
                                                    )]
        self.df['risk_factors'] = [[]] * len(self.df)
        self.df['match_indices'] = [[]] * len(self.df)
        for i in tqdm(range(len(risk_factors))):
            factor = risk_factors[i]
            if type(factor) == dict:
                name = factor['name']
                pattern = re.compile(factor['pattern'], re.IGNORECASE)
            elif type(factor) == str:
                name = factor
                pattern = re.compile(factor, re.IGNORECASE)
    
            self.df['_matches'] = self.df['text_body'].apply(lambda x: 
                                                                      [(m.start(), m.group()) 
                                                                       for m in pattern.finditer(x)])
            
            self.df[name + '_count'] = self.df['_matches'].apply(lambda x: len(x))
            self.df['has_' + name + '?'] = self.df[name + '_count'].apply(lambda x: x > 0)
            self.df[name + '_in_title'] = self.df['title'].apply(lambda x:
                                                                         len(pattern.findall(x)) > 0)
            self.df[name + '_count'] = self.df.apply(lambda x:
                                                             x[name + '_count'] + 10
                                                            if x[name + '_in_title'] 
                                                             else x[name + '_count'],
                                                            axis=1)
            self.df['total_rank'] += self.weights['risk'] * self.df[name + '_count']
            
            self.df['risk_factors'] = self.df.apply(lambda x: 
                                    x['risk_factors'] + [name] if x['has_' + name + '?']
                                    else x['risk_factors'],
                                   axis=1)
            self.df['match_indices'] = self.df.apply(lambda x: 
                                    x['match_indices'] + [n[0] for n in x['_matches']] if x['has_' + name + '?']
                                    else x['match_indices'],
                                   axis=1)
            self.df.drop('_matches', axis=1, inplace=True)
            

    def analyze_designs(self, design_list):
        """
        Analyses papers in self.df for study designs and returns a report df 
        with 'design' and 'design_rank'. 'design_rank' is decided upon from the 
        input in crowdsourced medical dictionary.
        """
        if design_list == None:
            design_list = PaperAnalyzer.DEFAULT_DESIGNS
        self.df['design'] = self.df['text_body'].progress_apply(lambda x:
                                                                      flatten([re.findall(des, x, re.IGNORECASE) 
                                                                       for des in design_list
                                                                       if re.findall(des, x, re.IGNORECASE)
                                                                      ]))
        self.df['design_rank'] = self.df['design'].apply(lambda x:
                                                                len(x))
        self.df['design_rank'] += self.df['design'].apply(lambda x: rank_design(' '.join(x)))
        self.df['total_rank'] += self.weights['design'] * self.df['design_rank']

    def analyze_outcomes(self, outcomes):
        """
        Analyses papers in self.df for outcomes and returns a report df 
        with 'outcomes' and 'outcome_rank'. 'outcome_rank' is decided upon 
        by the frequency of mentions of outcomes in the excerpt
        """
        
        if outcomes == None:
            outcomes = PaperAnalyzer.DEFAULT_OUTCOMES
        self.df['outcomes'] = self.df['text_body'].progress_apply(lambda x:
                                                                        flatten([re.findall(outc, x, re.IGNORECASE)
                                                                         for outc in outcomes
                                                                         if re.findall(outc, x, re.IGNORECASE)
                                                                        ]))
        self.df['outcome_rank'] = self.df['outcomes'].apply(lambda x: len(x))
        self.df['total_rank'] += self.weights['outcome']* self.df['outcome_rank']
        
    def analyze_fatality(self, fatality_list):
        """
        Analyses papers in self.df for information on fatality 
        returns a report df with 'fatality_rank'. 
        'fatality_rank' is decided upon by the frequency of 
        mentions of fatality in the excerpt
        """
        
        if fatality_list == None:
            fatality_list = PaperAnalyzer.DEFAULT_FATAL
        self.df['fatality_count'] = self.df['text_body'].progress_apply(lambda x:
                                                                        len([re.findall(key, x, re.IGNORECASE)
                                                                         for key in fatality_list]))
        self.df['fatal_info?'] = self.df['fatality_count'].apply(lambda x: x > 0)
        self.df['total_rank'] += self.weights['fatality']* self.df['fatality_count']

    def perform_analysis(self, risk_factors, design_list=None, outcomes=None, fatality_list = None):
        """
        This function is a wrapper function that provides interface
        to conduct analysis on all of risk factors, study designs and
        outcomes. Users may specify their own design_list or outcomes. If not,
        the default is used.
        """
        print("Analyzing risks")
        self.analyze_risks(risk_factors)
        print("Analyzing study designs")
        self.analyze_designs(design_list)
        print("Analyzing outcomes")
        self.analyze_outcomes(outcomes)
        print("Analyzing fatality")
        self.analyze_fatality(fatality_list)
        print("Generating final rankings")
        self.generate_risk_rankings()

        
    def generate_risk_rankings(self):
        """
        Appends columns in self.df that contain individual rankings for 
        each risk factor
        """
        self.df['section_rank'] = self.df['section'].apply(lambda x: self.section_ratings[x] 
                                                           if x in self.section_ratings else 5)
        # Obtaining list of risk factors
        risk_factors = [column for column in self.df.columns if 'has_' in column]
        risk_factors = [factor[4:-1] for factor in risk_factors]
        for i in tqdm(range(len(risk_factors))):
            factor = risk_factors[i]
            self.df[factor + '_rank'] = (
                            self.weights['risk']*self.df[factor + '_count'] + 
                            self.weights['section']*self.df['section_rank'] + 
                            self.weights['design']*self.df['design_rank'] + 
                            self.weights['outcome']*self.df['outcome_rank'] + 
                            self.weights['fatality']* self.df['fatality_count']
            )
            
            # Normalizing risk rank for length of excerpts
            self.df[factor + '_rank'] = self.df.apply(lambda x: x[factor + '_rank'] + 
                                                        (self.weights['inverse_length']/
                                                         (len(word_tokenize(x['text_body'])))), 
                                              axis=1)
        
        self.df['max_rank'] = self.df[[factor + '_rank' for factor in risk_factors]].max(axis=1)
        self.df['total_rank'] += self.weights['section'] * self.df['section_rank']
        # Normalizing total rank for length of excerpts
        self.df['total_rank'] = self.df.apply(lambda x: x['total_rank'] + 
                                                        (self.weights['inverse_length']/
                                                         (len(word_tokenize(x['text_body'])))), 
                                              axis=1)
    
    def get_df(self, risk_factor=None):
        """
        Applies section ratings, updates total ratings and returns reporting df
            risk_factor: if specified, the returned df will only have excerpts
                            that mention this risk factor
        """
        if risk_factor:
            if not self.__ANALYZED_RISKS:
                raise ValueError(self.__ERROR_MESSAGE)
            return self.df[self.df['has_' + risk_factor + '?'] == True]
        return self.df

In [405]:
covid_analysis = PaperAnalyzer(covid_df)

In [406]:
covid_analysis.analyze_risks(risk_factors)

In [407]:
covid_analysis.analyze_designs(design_list)

In [408]:
covid_analysis.analyze_outcomes(outcome_list)

In [409]:
covid_analysis.analyze_fatality(fatality_list)

In [410]:
covid_analysis.generate_risk_rankings()

In [411]:
temp = covid_analysis.get_df()
temp.shape

(661, 66)

In [412]:
[column for column in covid_analysis.get_df().columns if "has_" in column]

['has_smoking?',
 'has_diabetes?',
 'has_pregnancy?',
 'has_tuberculosis?',
 'has_hypertension?',
 'has_cancer?',
 'has_neonates?',
 'has_liver disease?',
 'has_COPD?',
 'has_pulmonary disease?',
 'has_race/ethnicity?']

In [423]:
temp = temp.sort_values(by='diabetes_rank', ascending=False).reset_index(drop=True)
#temp['design']
print(temp.loc[:,['title','text_body','diabetes_rank']].iloc[2]['text_body'])

Diabetes is a chronic disease, which requires frequent visits to the physician for lifestyle advice and adjustment of treatment. Telemedicine can help the patients to get in touch with their physician from the comfort of their home, and away from hospitals which could increase chances of coronavirus infections. Physicians can interact with the patient, gather history, analyze their self-monitored blood glucose charts (SMBG) and self-monitored blood pressure (SMBP) values and give advice.
The guidelines given by MOHFW 7 suggest video mode of communication for first consult however, in our opinion the first consult for patients with diabetes should ideally be face-to-face if possible, because physical examination cannot be replaced by telemedicine unless another healthcare worker in physical proximity to the patient can relay examination findings.
Scientific studies support the usage of telemedicine in patients of diabetes. A metaanalysis of 35 randomized controlled trials (RCTs) of tele

In [433]:
# There are some papers that appear more than once. This is because we rank excerpts within papers, and could simply mean that two different excerpts within the same paper were ranked high in relevance by our algorithm.
smoking_df = enriched_covid_df.sort_values(by='smoking_rank', ascending=False).loc[:,['title','doi','publish_time','design','text_body']].reset_index(drop=True).rename(columns={'text_body':'excerpt'})
#smoking_df.to_csv('smoking_risk_analysis.csv')
smoking_df.head()

,title,doi,publish_time,design,excerpt
0,Bulk and single-cell transcriptomics identify ...,10.1101/2020.02.05.20020107,2020-02-11,"[statist, statist, analysi, analysi, analysi, ...",We found a significant higher ACE2 gene expres...
1,"Prevalence, Severity and Mortality associated ...",10.1101/2020.03.25.20043745,2020-03-27,"[model, analysi, investig, systematic review ,...","unknown, and undoubtedly more research is need..."
2,Bulk and single-cell transcriptomics identify ...,10.1101/2020.02.05.20020107,2020-02-11,"[model, model, model, analysi, analysi]",Duclos G et.al. studied human bronchial epithe...
3,Bulk and single-cell transcriptomics identify ...,10.1101/2020.02.05.20020107,2020-02-11,"[model, statist, investig]","In this study, we investigated the disparities..."
4,"Prevalence, Severity and Mortality associated ...",10.1101/2020.03.25.20043745,2020-03-27,[],The pooled prevalence of COVID-19 patients who...


In [443]:
# Here is a title-deduplicated version of the previous dataframe
# Each paper will only appear once with its highest rated excerpt
smoking_df.drop_duplicates('title', keep='first').reset_index(drop=True).head()

,title,doi,publish_time,design,excerpt
0,Bulk and single-cell transcriptomics identify ...,10.1101/2020.02.05.20020107,2020-02-11,"[statist, statist, analysi, analysi, analysi, ...",We found a significant higher ACE2 gene expres...
1,"Prevalence, Severity and Mortality associated ...",10.1101/2020.03.25.20043745,2020-03-27,"[model, analysi, investig, systematic review ,...","unknown, and undoubtedly more research is need..."
6,): e35 REVIEW ARTICLE A. Prevalence of Underly...,NaN,2020-03-24,"[contact, contact, contact, contact, model, ou...",China and the rest of the world have faced an ...
7,Articles Clinical and epidemiological features...,10.1016/s1473-3099(20)30198-5,2020-03-25,[],Pulmonary ground-glass opacities 6 (60%) 13 (5...
15,The COVID-19 pandemic and the use of MS diseas...,10.1016/j.msard.2020.102073,2020-03-27,"[profil, profil, contact, contact, analysi]",It is clear that COVID-19 is a pandemic and gl...


In [445]:
diabetes_df = enriched_covid_df.sort_values(by='diabetes_rank', ascending=False).loc[:,['title','doi','publish_time','design','text_body']].reset_index(drop=True).rename(columns={'text_body':'excerpt'})
#diabetes_df.to_csv('smoking_risk_analysis.csv')

In [447]:
# Each paper will only appear once with its highest rated excerpt
diabetes_df.drop_duplicates('title', keep='first').reset_index(drop=True).head()

,title,doi,publish_time,design,excerpt
0,Comorbid Diabetes Mellitus was Associated with...,10.1101/2020.03.24.20042358,2020-03-26,"[model, investig, retrospective cohort]","In this retrospective cohort study, we charact..."
2,Journal Pre-proof Telemedicine for Diabetes Ca...,10.1016/j.dsx.2020.04.001,2020-04-04,"[analysi, randomized controlled]","Diabetes is a chronic disease, which requires ..."
5,Exploring diseases/traits and blood proteins c...,10.1101/2020.03.04.20031237,2020-03-08,"[statist, analysi, analysi, analysi, analysi, ...","Remarkably, a number of top-ranked results wer..."
8,Articles Clinical and epidemiological features...,10.1016/s1473-3099(20)30198-5,2020-03-25,[],Pulmonary ground-glass opacities 6 (60%) 13 (5...
17,Acute liver injury and its association with de...,10.1101/2020.04.02.20050997,2020-04-06,"[logistic regression, analysi, investig]",The present study aimed to analyze SARS-CoV-2-...


In [448]:
pregnancy_df = enriched_covid_df.sort_values(by='pregnancy_rank', ascending=False).loc[:,['title','doi','publish_time','design','text_body']].reset_index(drop=True).rename(columns={'text_body':'excerpt'})
#pregnancy_df.to_csv('smoking_risk_analysis.csv')

In [450]:
# Each paper will only appear once with its highest rated excerpt
diabetes_df.drop_duplicates('title', keep='first').reset_index(drop=True).head()

,title,doi,publish_time,design,excerpt
0,Comorbid Diabetes Mellitus was Associated with...,10.1101/2020.03.24.20042358,2020-03-26,"[model, investig, retrospective cohort]","In this retrospective cohort study, we charact..."
2,Journal Pre-proof Telemedicine for Diabetes Ca...,10.1016/j.dsx.2020.04.001,2020-04-04,"[analysi, randomized controlled]","Diabetes is a chronic disease, which requires ..."
5,Exploring diseases/traits and blood proteins c...,10.1101/2020.03.04.20031237,2020-03-08,"[statist, analysi, analysi, analysi, analysi, ...","Remarkably, a number of top-ranked results wer..."
8,Articles Clinical and epidemiological features...,10.1016/s1473-3099(20)30198-5,2020-03-25,[],Pulmonary ground-glass opacities 6 (60%) 13 (5...
17,Acute liver injury and its association with de...,10.1101/2020.04.02.20050997,2020-04-06,"[logistic regression, analysi, investig]",The present study aimed to analyze SARS-CoV-2-...


In [451]:
hypertension_df = enriched_covid_df.sort_values(by='hypertension_rank', ascending=False).loc[:,['title','doi','publish_time','design','text_body']].reset_index(drop=True).rename(columns={'text_body':'excerpt'})
#hypertension_df.to_csv('smoking_risk_analysis.csv')

In [452]:
# Each paper will only appear once with its highest rated excerpt
hypertension_df.drop_duplicates('title', keep='first').reset_index(drop=True).head()

,title,doi,publish_time,design,excerpt
0,Angiotensin II Receptor Blockers and Angiotens...,10.1101/2020.03.31.20038935,2020-04-04,[],"After the initial screen, 462 COVID-2019 patie..."
5,Articles Clinical and epidemiological features...,10.1016/s1473-3099(20)30198-5,2020-03-25,[],Pulmonary ground-glass opacities 6 (60%) 13 (5...
7,The COVID-19 pandemic and the use of MS diseas...,10.1016/j.msard.2020.102073,2020-03-27,"[profil, profil, contact, contact, analysi]",It is clear that COVID-19 is a pandemic and gl...
8,Acute liver injury and its association with de...,10.1101/2020.04.02.20050997,2020-04-06,"[logistic regression, analysi, investig]",The present study aimed to analyze SARS-CoV-2-...
9,Comorbid Diabetes Mellitus was Associated with...,10.1101/2020.03.24.20042358,2020-03-26,[model],To further assess the association of diabetes ...


In [453]:
tuberculosis_df = enriched_covid_df.sort_values(by='tuberculosis_rank', ascending=False).loc[:,['title','doi','publish_time','design','text_body']].reset_index(drop=True).rename(columns={'text_body':'excerpt'})
#tuberculosis_df.to_csv('smoking_risk_analysis.csv')

In [456]:
# Each paper will only appear once with its highest rated excerpt
tuberculosis_df.drop_duplicates('title', keep='first').reset_index(drop=True).head()

,title,doi,publish_time,design,excerpt
0,Active or latent tuberculosis increases suscep...,10.1101/2020.03.10.20033795,2020-03-16,[],The copyright holder for this preprint . https...
1,Articles Clinical and epidemiological features...,10.1016/s1473-3099(20)30198-5,2020-03-25,[],Pulmonary ground-glass opacities 6 (60%) 13 (5...
2,Comorbid Diabetes Mellitus was Associated with...,10.1101/2020.03.24.20042358,2020-03-26,[model],To further assess the association of diabetes ...
3,The COVID-19 pandemic and the use of MS diseas...,10.1016/j.msard.2020.102073,2020-03-27,"[profil, profil, contact, contact, analysi]",It is clear that COVID-19 is a pandemic and gl...
4,viruses Perspective Potential Maternal and Inf...,10.3390/v12020194,2020,[outbreak],"At times of SARS outbreaks, all pregnant patie..."


In [414]:
enriched_covid_df = covid_analysis.get_df()

In [397]:
enriched_covid_df.shape

(661, 66)

In [225]:
#enriched_covid_df.sort_values(by='smoking_rank', ascending=False).iloc[0]['text_body']

In [22]:
enriched_covid_df.to_json("../../enriched_covid_df.json", orient='records')
enriched_covid_df.to_csv("../../enriched_covid_df.csv")

#### This marks the end of phase 1. The resulting dataframe will be stored as a json to be served by the web app.

<a id="phase2"></a>
## Question Search (Phase 2)
We will extend the capabilities from the `PaperAnalyzer` class and attempt to answer some questions.

### The `Question` class will decompose and resolve a question about risk factors.
The result will then be piped to an instance of `PaperAnalyzer` to conduct similar analysis. Users will be able to specify their own list of outcomes. If not specified, the default set of outcomes will be used.

In [23]:
class Question():
    """
    The purpose of this class is to resolve a question for 
    keyword searching
    """
    def __init__(self, question, design_list=None, outcomes=None):
        """
        The constuctor does most of the method-calling for question resolution
        """
        self.DESIGN_LIST = design_list
        self.OUTCOMES = outcomes
        self.RISK = question
        self.risk_factors = None
        self.design_list = None
        self.outcome_list = None
        self.__resolve_question()
        if design_list:
            self.__resolve_design()
        if outcomes:
            self.__resolve_outcomes()

    def __question_tokenize(self, sent):
        """
        Cleans the question string
        """
        abbvr_pattern = re.compile('(e.g.|i.e.)')
        sent = abbvr_pattern.sub('', sent)
        remove_punct_dict = {key: " " for key in string.punctuation}
        remove_punct_dict['.'] = ''
        remove_punct = str.maketrans(remove_punct_dict)
        sent = sent.translate(remove_punct)
        return sent.replace('R', 'R0').replace('-', ' ')

    def __resolve_question(self):
        """
        stems and removes irreleavnt words from questions
        to create keywords for keyword analysis
        """
        subquestion = self.RISK
        sub_q = self.__question_tokenize(subquestion)
        keywords = set([
            stemmer.stem(word) for word in word_tokenize(sub_q)
            if word.lower() not in stop_words and 'cov' not in word.lower()
            and word.lower().islower()  #This checks and removes numbers
        ])
        self.risk_factors = list(keywords)

    def __resolve_design(self):
        """
        Resolves study designs to allow for study-design evaluation
        """
        design_keys = self.DESIGN_LIST.split(",")
        self.design_list = list(set(design_keys))

    def __resolve_outcomes(self):
        """
        Resolves outcomes to allow for outcome evaluation
        """
        outcome_keys = self.__question_tokenize(self.OUTCOMES)
        outcome_keys = set([
            stemmer.stem(word) for word in word_tokenize(outcome_keys)
            if word.lower() not in stop_words
            and word.lower().islower()  #This checks and removes numbers
        ])
        self.outcome_list = list(outcome_keys)

    def get_keywords(self):
        """
        Returns keywords from earlier methods
        """
        result = {'risk': None, 'design': None, 'outcome': None}
        result['risk'] = self.risk_factors
        if self.design_list:
            result['design'] = self.design_list
        if self.outcome_list:
            result['outcome'] = self.outcome_list
        return result

In [24]:
def analyze_question(df, question):
    """
    Function to take in a Question instance and a 
    dataframe with covid-excerpts to perform 
    evaluation and rankings on information relevancy
    """
    reference_df = PaperAnalyzer(df)
    keys = question.get_keywords()
    reference_df.perform_analysis(keys['risk'], keys['design'],
                                  keys['outcome'])
    return reference_df.get_df()

#### We will be trying out the question answering pipeline now with a few questions from the aforementionned [medical dictionary](https://docs.google.com/spreadsheets/d/1t2e3CHGxHJBiFgHeW0dfwtvCG4x0CDCzcTFX7yz9Z2E/edit#gid=1217643351).

In [25]:
def get_google_sheet(url, sheet_name):
    response=requests.get(url=url)
    sample_file = io.BytesIO(response.content)
    df = pd.read_excel(sample_file, sheet_name = sheet_name)
    return df

dict_url = 'https://docs.google.com/spreadsheets/d/1t2e3CHGxHJBiFgHeW0dfwtvCG4x0CDCzcTFX7yz9Z2E/export?format=xlsx&id=1t2e3CHGxHJBiFgHeW0dfwtvCG4x0CDCzcTFX7yz9Z2E'
questions_df = get_google_sheet(dict_url, 'sub.question.matching')

In [26]:
questions_df.head(2)

,Unnamed: 0,Question,Subquestion,Outcome.list,Differences.list,Design.list,Notes
0,"What is known about transmission, incubation, ...",Range of incubation periods for the disease in...,Range of incubation periods for humans: genera...,incubation period,age.,"contact tracing, survival analysis, time-to-ev...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...
1,"What is known about transmission, incubation, ...",Range of incubation periods for the disease in...,Range of incubation periods for humans: by age...,"incubation period, stratified by age group",age.,"contact tracing, survival analysis, time-to-ev...",Notes


In [27]:
# Picking a sample question for analysis
ques = questions_df.iloc[127]['Subquestion']
ques = sent_tokenize(ques)[0]
ques

'Is COVID-19 transmitted on droplets?'

In [28]:
# Designs recommended for sample question
des = questions_df.iloc[127]['Design.list']
des

'risk factor analysis, logistic regression, cross-sectional case-control, matched case-control, observational case series, time series analysis, survival analysis'

In [29]:
# Outcomes recommended for sample question
outc = questions_df.iloc[127]['Outcome.list']
outc

'odds of COVID-19 acquisition by occupation, age group, PPE use, observed/self-reported risk behaviors (e.g., inappropriately lowering mask to speak, touching face, eating without washing hands)'

In [30]:
report_df = analyze_question(covid_df, Question(ques, des, outc))

Analyzing risks



Analyzing study designs



Analyzing outcomes



Analyzing fatality



Generating final rankings


#### We will now look at the 5 most relevant excerpts to the question we picked earlier

In [31]:
for i in range(5, 10):
    print(report_df.sort_values(by='droplet_rank', ascending = False).iloc[i]['text_body'][:1000])
    print("-----------------------------","\n")

Intubating a patient with COVID-19 is a high-risk procedure, due to the proximity of the health care workers to the patients' oropharynx and the exposure to airway secretions, which can carry a high viral load. 47 During the SARS outbreak in 2003, health care workers performing intubations were shown to be at a significantly increased risk of nosocomial transmission. 48 This risk was shown to be greatly reduced where PPE was used appropriately and infection control measures were followed. 10 The availability and suitability of facemasks and respirators has escalated into an emotive, as well as scientific debate. A fluid resistant surgical facemask protects the wearer against sprays of bodily fluids and large droplets, whereas N95, FFP2 and FFP3 respirators are thought to protect the wearer against aerosolised and airborne pathogens as well. In laboratory studies, a FFP2 mask filters at least 94% of all particles that are 0.3 microns in diameter or larger; N95 masks block at least 95%, 